In [110]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder\
                    .appName('task-4')\
                    .getOrCreate()

df = spark.read.format('org.apache.spark.sql.json')\
                .option("recursiveFileLookup","true")\
                .option("header",'true')\
                .load('/Users/rikinpatel/open-data-master/data/matches/',multiline = True)

In [121]:
occurences = df.select('home_team.home_team_name','away_team.away_team_name','season.season_name')

occurences = occurences.drop('away_team_name')\
                       .union(occurences\
                       .select('away_team_name','season_name'))\
                       .withColumnRenamed('home_team_name','Teams')

result = occurences.groupBy('Teams')\
                       .agg(count('season_name'))\
                       .orderBy('count(season_name)',ascending= False)

print('{} had the most number of appearances across all seasons :'.format(result.collect()[0][0]),':',result.collect()[0][1])

Barcelona had the most number of appearances across all seasons : : 455


In [122]:
occurences = occurences.crosstab('Teams','season_name')

for cols in occurences.columns :
    occurences = occurences.filter(col(cols) != 0)

print('{} teams appeared in each and every season!'.format(occurences.count()))

0 teams appeared in each and every season!
